In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import randint,uniform,choice

import time

# General Parameters

In [2]:
n_pop = 10000
n_loc = 2601
n_net = 200
n_overlap = 100

# |  L  |  L  |  L  |  L  |  L  |  L  |  L  |  L  | 
# |  N  |  N  |  N  | *N  | *N  |  N  |  -  |  -  | 
# |  -  |  -  |  -  | *H  | *H  |  H  |  H  |  H  |
# ___Only Netorks___  (*OVERLAP) _______Homes_________


labels = [0,1,2,3,4] # Number of states a person can be in, [S,E,I,R,D] (Not very important, just for bookkeeping)

person_attr = 4  # 4 attributes for a person: state, home, network, current location

### Rates #############

transmit_rate = 0.2
e_to_i_rate   = 0.2
recovery_rate = 0.1
death_fraction= 0.01

random_test_rate =0.1
test_sensitivity =0.99
finish_test_rate =1.0

# Step 1: Create a population

Individuals are defined by `['state', 'home_location', 'network_location', 'current_location' ]`, for example: 

`[0, 0, 4, 0]` is an individual who is susceptible, whose home is `0` and whose work is `4`, and who is currently at home.

In [3]:
def create_person(state, home):                  # Function to create a person with certain attributes
    net = randint(0,n_net)                       # Assign a random network from 0 to n_net (here, 200)
    return np.array([state, home , net , home ]) # Create a person in state 'state', initially at home


pop = np.zeros((n_pop,person_attr),int)          # Empty population
link_weight = np.zeros(n_pop,float)              # Array for link weights
is_confined = np.zeros(n_pop,bool)               # Boolean array, is confined? (Initial values, all "no".)
is_dead     = np.zeros(n_pop,bool)               # Boolean array, is dead? (Initial values, all "no".)
being_tested= np.zeros(n_pop,bool)               # Boolean array, is being tested?

n_per_location=np.zeros((n_loc,len(labels)),int) # Array to hold total number of individuals per state 
                                                 # for all locations: both homes and networks

# Initial populations ###########

n_inf = 10
n_exp = 0
n_rec = 0
n_dead= 0
n_sus = n_pop - n_inf

output = np.array([0,n_sus, n_exp, n_inf, n_rec, n_dead],int)  # Output of the form t,S,E,I,R,D

##################################


for i in range(0,n_pop):
    home = randint(n_overlap-1,n_loc)              # Assign random homes for remainder with random network
    pop[i] = create_person(0,home)

    n_per_location[  pop[i][3] ][0] +=  1          # Increment susceptibles in each person's location

    link_weight[i] = 0.1

r = choice(n_pop,size=n_inf,replace=False)         # Generate n_inf random numbers (r) from the total population without replacement

np.put_along_axis(pop[:,0],r,2,axis=0)             # Set population members at r to infected (2)

i_locs = pop[r][:,3]                               # Find locations of infected individuals (with repetition), eg. [1,2,1,1,3,1,2,3..] etc

np.add.at(n_per_location[:,0],i_locs,-1)           # Decrement number of infected at these locations (repetition increments twice)
np.add.at(n_per_location[:,2],i_locs,+1)           # Increment number of susceptible at these locations (idem)

if(np.sum(n_per_location) != n_pop):
    print("Error: Problem with assigning infected individuals to location. Mismatch in total population number.")


# Step 2: Run Gillespie

In [4]:
def Gillespie_Run(tf,output):
    
    n_sus = output[1]
    n_exp = output[2]
    n_inf = output[3]
    n_rec = output[4]
    n_dead= output[5]

    t = 0.0
    t_out = 0
    
    while(n_exp+n_inf>0):   # Until there's no one left to infect anyone (could be t<tf too.)
            
        if(t>=t_out):
            output = np.vstack((output,[t_out,n_sus,n_exp,n_inf,n_rec,n_dead]))
            t_out += 1
            print(t)
        
        
        is_not_conf_or_dead = np.logical_not(is_confined+is_dead)   # Boolean array of people neither confined nor dead 
        Si_Ii = n_per_location[:,0]*n_per_location[:,2]             # Array of Si x Ii

        a1 = transmit_rate*np.sum(Si_Ii)        
        a2 = a1 + n_exp*e_to_i_rate
        a3 = a2 + n_inf*recovery_rate
        a4 = a3 + np.sum(is_not_conf_or_dead)*random_test_rate           # If the rates become person specific, move the product inside the sum
        a5 = a4 + np.sum(being_tested)*finish_test_rate                  # idem
        
        a0 = a5 + np.sum(is_not_conf_or_dead*link_weight)

        # The above values aren't the cumulative values, so I handle that below

        dt = -np.log(uniform(0.0, 1.0)) / a0          # Pick the next time step
        t = t + dt                                    # increment time

        a = uniform(0.0,1.0) * a0                     # Choose which process occurs

        
        if(a<a1):                                     # In this case, an S-> E
            psum = 0
        
            for i in range(0,n_pop):
                if pop[i][0] == 0:
                    psum += transmit_rate * n_per_location[ pop[i][3] ][2]

                    if (psum>a):
                        break
            
            pop[i][0]=1                                # Set that person to exposed
            
            n_exp = n_exp + 1                          # Increment number of exposed
            n_sus = n_sus - 1                          # Decrement number of susceptible
            
            n_per_location[pop[i][3]][1] += 1          # Increment exposed in location of person
            n_per_location[pop[i][3]][0] -= 1          # Decrement susceptibles in location of person
            

        elif(a<a2):                                    # Here E-> I
            psum = a1
            
            for i in range(0,n_pop):
                if(pop[i][0]==1):
                    psum += e_to_i_rate

                    if (psum>a):
                        break;

            pop[i][0]=2
            
            n_inf = n_inf + 1                          # Increment number of infected
            n_exp = n_exp - 1                          # Decrement number of exposed
                  
            n_per_location[pop[i][3]][2] += 1          # Increment infected in location of person
            n_per_location[pop[i][3]][1] -= 1          # Decrement exposed in location of person


        elif(a<a3):                                    # I -> R || D
            psum = a2

            for i in range(0,n_pop):

                if(pop[i][0]==2):
                    psum += recovery_rate

                    if(psum>a):
                        break

            is_confined[i] = True
 

            if(uniform(0.0,1.0)<death_fraction):        # Testing if person died or recovered
                pop[i][0]=4                             # Oh no! Dead!

                n_dead= n_dead+ 1                       # Increment number of recovered
                n_inf = n_inf - 1                       # Decrement number of infected

                n_per_location[pop[i][3]][4] += 1       # Increment dead in location of person
                n_per_location[pop[i][3]][2] -= 1       # Decrement infected in location of person
                
                is_dead[i] = True                       # Set the person to being dead
                
            else:
                pop[i][0]=3                             # Yay! Recovered!
                
                n_rec = n_rec + 1                       # Increment number of recovered
                n_inf = n_inf - 1                       # Decrement number of infected

                n_per_location[pop[i][3]][3] += 1       # Increment recovered in location of person
                n_per_location[pop[i][3]][2] -= 1       # Decrement infected in location of person

        elif(a<a4):                                     # Conduct a random test
            psum = a3

            for i in range(0,n_pop):
                if(not is_confined[i] and not is_dead[i]):# If the person isn't confined or dead
                    psum += random_test_rate
                    if(psum>a):
                        break

            being_tested[i]=True

        elif(a<a5):
            psum = a4
            for i in range(0,n_pop):

                if being_tested[i] == True:
                    psum += finish_test_rate

                    if(psum>a):
                        break

            being_tested[i]=False    

            if (pop[i][0]==2 and uniform(0.0,1.0)<test_sensitivity): # If the person is infected and the test works
                is_confined[i] = True                                # They are confined
                
                n_per_location[pop[i][3]][2] -= 1          # Decrement infected in current (NET or HOME) location of person
                pop[i][3] = pop[i][1]                      # Send them home (set current location to home location)
                n_per_location[pop[i][3]][2] += 1          # Increment infected in HOME location of person


        else:                                              # All else failing, move people around
            psum = a5
            for i in range(0,n_pop):

                if (not is_confined[i] and not is_dead[i]):  # If they aren't confined or dead
                    psum += link_weight[i]                 # move them to their other location

                    if(psum>a):
                        break

            home_loc = pop[i][1]
            net_loc  = pop[i][2]

            if(pop[i][3]==home_loc):                       # If the person is at home
                pop[i][3] = net_loc                        # move them to their network location
                
                for j in range(0,len(labels)):
                    if(pop[i][0]==j):
                        n_per_location[home_loc][j] -= 1
                        n_per_location[net_loc][j] += 1     
                
            else:
                pop[i][3] = home_loc                       # otherwise move them home
                
                for j in range(0,len(labels)):
                    if(pop[i][0]==j):
                        n_per_location[net_loc][j] -= 1
                        n_per_location[home_loc][j] += 1

    return output


In [5]:
### Running the Code #### 

start = time.time()

to_print_to_file = Gillespie_Run(100,output)

end = time.time()

print("Time elapsed: "+str(end-start))

np.savetxt("SEIRD_with_testing.txt",to_print_to_file,delimiter=" ")


##########################

0.0
1.0002491076859144
2.0000540804626565
3.0005637077543317
4.000117852447005
5.000167318720603
6.00065784914382
7.000008805156092
8.000077206705672
9.000068223337344
10.000581326503621
11.000348050555631
12.00001177021539
13.000477834131145
14.000037062060137
15.000133500856931
16.000319157148002
17.000164289764424
18.000255231898297
19.00003210174764
20.000091654409687
21.00005647499939
22.000533815890414
23.00006732090817
24.00047360291507
25.000041482357144
26.000132521422934
27.000869597254383
28.00067790718316
29.000027493094994
30.000263579556865
31.000010491361824
32.000287662732475
33.00033048140271
34.00006001864692
35.00010661328609
36.00028137324969
37.00082438505633
38.000171353140225
39.00002838309464
40.00104633324084
41.00036775625824
42.00001872677754
43.00008433636285
44.00126544576533
45.00083217804678
46.0002330700104
47.000166259141025
48.000615769528245
49.00048263067972
50.00628474360395
51.004952829298034
52.00233060696123
53.00030248797328
54.00096827938463
55